In [1]:
import random
import pandas as pd
import numpy as np

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation

In [116]:
class Sensor(Agent):
    
    def __init__(self, unique_id, battery_life, 
                record_cost, record_freq, record_bytes, 
                compute_cost, sign_cost,
                transmit_cost, transmit_freq,
                blockchain, model):
        
        super().__init__(unique_id, model)
           
        self.unique_id = unique_id
        self.battery_life = battery_life
        self.record_cost = record_cost
        self.record_freq = record_freq
        self.record_bytes = record_bytes
        self.compute_cost = compute_cost
        self.sign_cost = sign_cost
        self.transmit_cost = transmit_cost
        self.transmit_freq = transmit_freq
        self.blockchain = blockchain
        self.gwei_spent = 0

    
    def record(self):

        if self.model.schedule.steps % self.record_freq == 0:
            
            # STOCHASTICITY vv
            self.battery_life -= self.record_cost
        else:
            pass
    
    def compute(self):
        
        # STOCHASTICITY vv
        self.battery_life -= self.compute_cost
    
    def sign(self):
        
        self.battery_life -= self.sign_cost
    
    def transmit(self, num_bytes):
        
        # STOCHASTICITY vv
        self.battery_life -= self.transmit_cost
        self.gwei_spent += self.blockchain.write_data(num_bytes)
        self.gwei_spent += self.blockchain.compute(2)
    
    def step(self):
        
        if self.battery_life < 0 and self.battery_life is not -999:
            if self.model.verbose:
                print("Sensor", self.unique_id, "out of battery at tick", self.model.schedule.steps)
            self.model.schedule.remove(self)
        
        if self.model.schedule.steps % self.record_freq == 0:
            self.record()    
    
        if self.model.schedule.steps % self.transmit_freq == 0:
            # Work out how many bytes to transmit
            
            # STOCHASTICITY vv
            bytes_to_transmit = 32 # <-- calc with record_bytes, transmit freq
            self.sign()
            self.transmit(bytes_to_transmit)

        


In [150]:
class Blockchain(Agent):
    
    def __init__(self, unique_id, gas_price,
                gas_per_byte, gas_per_second, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print("Blockchain created: ID", unique_id)
        
        self.unique_id = unique_id
        self.gas_price = gas_price
        self.gas_per_byte = gas_per_byte
        self.gas_per_second = gas_per_second
        self.chain_size = 0
    
    def write_data(self, num_bytes):
        self.chain_size += num_bytes
        gwei_spent = self.gas_per_byte * num_bytes * self.gas_price
        return gwei_spent

    def compute(self, num_seconds):
        gwei_spent = self.gas_per_second * num_seconds * self.gas_price
        return gwei_spent

In [153]:
class SensorBlockchainNetwork(Model):
    
    def __init__(self, num_sensors,
                verbose=True):
        
        super().__init__()
        
        self.verbose = verbose
        if self.verbose:
            print('Verbose model')
        
        self.running = True
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
                                model_reporters = {
                                    "active_sensors": lambda m: len(m.schedule._agents),
                                    "blockchain_size": lambda m: m.blockchain.chain_size
                                },
                                agent_reporters = {
                                    "gwei_spent": lambda a: a.gwei_spent,
                                    "battery_life": lambda a: a.battery_life
                                })
        
        self.blockchain = Blockchain(self.next_id(), 10,
                                    625, 75000000, self) 
                                    # calculated based on 
                                    # https://hackernoon.com/ether-purchase-power-df40a38c5a2f
        
        for i in range(num_sensors):
            sensor = Sensor(self.next_id(), 100, 
                            1, 1, 32,
                            1, 1,
                            1, 1,
                            self.blockchain,
                            self)
            
            self.schedule.add(sensor)
    
        if self.verbose:
            print(num_sensors, "instantiated and added to schedule.")
    
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [155]:
model = SensorBlockchainNetwork(100, False)

In [156]:
for i in range(10):
    model.step()

In [157]:
a_df = model.datacollector.get_agent_vars_dataframe()
m_df = model.datacollector.get_model_vars_dataframe()

In [159]:
m_df

,active_sensors,blockchain_size
0,100,3200
1,100,6400
2,100,9600
3,100,12800
4,100,16000
5,100,19200
6,100,22400
7,100,25600
8,100,28800
9,100,32000


In [ ]:
# Batch Run

batch_runner = BatchRunner(
    SensorBlockchainNetwork,
    variable_params,
    fixed_params,
    iterations = 5,
    max_steps = 100,
    model_reporters = {"blockchain_size": lambda m: m.blockchain.chain_size}
)